In [1]:
import pyaudio,wave
import ollama
import warnings
import os
import glob
import sys
import torch
import re
import logging
os.environ['HF_ENDPOINT']='hf-mirror.com'#hugging_face镜像
# os.environ['HF_HUB_OFFLINE']='0'

#清理内存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
if hasattr(torch.cuda, 'empty_cache'):
    torch.cuda.empty_cache()
del torch
warnings.filterwarnings("ignore")

original_path = sys.path

In [2]:
# #pyttsx3文转语音
# import pyttsx3
# sys.path.append('../text_to_voice')
# from pyttsx3_def import text_voice##
# engine = pyttsx3.init()
# engine = text_voice(engine,rate=130)
# (r,ve,vs) = (engine.getProperty('rate'),engine.getProperty('volume'), engine.getProperty('voices'))
# print('目前语速是：',r,'\t','语音音量是:',ve)

#导入语音合成模型
#选择运行哪一个版本
version_choose = '0306fix' 
version_choose = 'fast_inference'

if version_choose == 'fast_inference' :
    from GPT_SoVITS.inference_webui_fast_inference_maple import handle

elif version_choose == '0306fix':
    from GPT_SoVITS.inference_maple import handle

---------------------------------------------TTS Config---------------------------------------------
device              : cuda
is_half             : True
t2s_weights_path    : GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt
vits_weights_path   : GPT_SoVITS\pretrained_models\s2G488k.pth
bert_base_path      : GPT_SoVITS\pretrained_models\chinese-roberta-wwm-ext-large
cnhuhbert_base_path : GPT_SoVITS\pretrained_models\chinese-hubert-base
flash_attn_enabled  : True
----------------------------------------------------------------------------------------------------

Loading Text2Semantic weights from GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt
Using Flash Attention
Loading VITS weights from GPT_SoVITS\pretrained_models\s2G488k.pth
Loading BERT weights from GPT_SoVITS\pretrained_models\chinese-roberta-wwm-ext-large
Loading CNHuBERT weights from GPT_SoVITS\pretrained_models\chinese-hubert-base


In [3]:
#声音转文本模型加载
model_choose='sherpa_onnx'#fast_whisper, whisper, sherpa_onnx
# model_choose='faster_whisper'
# model_choose='whisper'

#获取LargeModel所在的路径（事先将它添加到PYTHONPATH的环境变量中）
path= [content for content in sys.path if re.match('.+:.*LargeModel$', content)][0]

if model_choose=='faster_whisper':
    from voice_record.voice_record_def import listen#记录输入的语音
    from Speech_to_Text.Fast_whisper.Fast_whisper import fast_whisper_opt
    from faster_whisper import WhisperModel

    model_size = "small"
    # model_size='large-v3'
    download_root=path+'/Speech_to_Text/fast_whisper'
    #当download_root与模型在同一个目录时候
    model = WhisperModel(download_root+'\\'+model_size, device="cuda", compute_type="float16", download_root=download_root, local_files_only=False)

#whisper十分垃圾
elif model_choose=='whisper':
    # #whisper
    import whisper
    from voice_record.voice_record_def import listen#记录输入的语音
    from Speech_to_Text.Whisper.Whisper import whisper_output
    model_path = path+'/Speech_to_Text/Whisper/Model'
    # model= whisper.load_model(name='small', download_root=model_path)#device='cuda:0'
    model= whisper.load_model(name='small', download_root=model_path)#device='cuda:0'

elif model_choose == 'sherpa_onnx':
    import sherpa_onnx
    from kaldi.sherpa_onnx_model.stream_zipformer import microphone_endpoint
    sherpa_path = path+'/'+'kaldi/sherpa_onnx_model/stream_zipformer'
    args_dict = {  
        'tokens': sherpa_path+'/tokens.txt',  
        'encoder': sherpa_path+'/encoder-epoch-99-avg-1.onnx',  
        'decoder': sherpa_path+'/decoder-epoch-99-avg-1.onnx' ,  
        'joiner': sherpa_path+'/joiner-epoch-99-avg-1.onnx',    
    } 
    #检查设备，顺便输出recongnizer
    check_recognizer= microphone_endpoint.main(args_dict)
else:
    print('Choose unfound asr model！')


#文本对话初始化
text_text_model='qwen:4b-chat-v1.5-q6_K'

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 耳机 (EDIFIER W820NB Hands-Free A, MME (2 in, 0 out)
   2 麦克风阵列 (适用于数字麦克风的英特尔® 智音技术), MME (2 in, 0 out)
   3 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  4 耳机 (EDIFIER W820NB Stereo), MME (0 in, 2 out)
   5 扬声器 (2- Realtek(R) Audio), MME (0 in, 2 out)
   6 耳机 (EDIFIER W820NB Hands-Free A, MME (0 in, 2 out)
   7 主声音捕获驱动程序, Windows DirectSound (2 in, 0 out)
   8 耳机 (EDIFIER W820NB Hands-Free AG Audio), Windows DirectSound (2 in, 0 out)
   9 麦克风阵列 (适用于数字麦克风的英特尔® 智音技术), Windows DirectSound (2 in, 0 out)
  10 主声音驱动程序, Windows DirectSound (0 in, 2 out)
  11 耳机 (EDIFIER W820NB Stereo), Windows DirectSound (0 in, 2 out)
  12 扬声器 (2- Realtek(R) Audio), Windows DirectSound (0 in, 8 out)
  13 耳机 (EDIFIER W820NB Hands-Free AG Audio), Windows DirectSound (0 in, 1 out)
  14 扬声器 (2- Realtek(R) Audio), Windows WASAPI (0 in, 8 out)
  15 耳机 (EDIFIER W820NB Stereo), Windows WASAPI (0 in, 2 out)
  16 耳机 (EDIFIER W820NB Hands-Free AG Audio)

In [4]:
#语音合成模型初始化
ref_wav_path=r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\派蒙-效果并不好\参考音频\说话—既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。.wav"
ref_wav_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\面试官这边临时有改动，然后，明天下午两点可以吗？.wav"
ref_wav_path =r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\这是一个传承了千年尊贵的名字。.wav"
prompt_text= '既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。'
prompt_text = "面试官这边临时有改动，然后，明天下午两点可以吗？"
prompt_text = "这是一个传承了千年，尊贵的名字。"
prompt_language='zh'
text='晚上好，祝你有一个甜美的睡眠！'#'你好呀！'
text_language='zh'
ref_free=False
start=False
top_k=5
top_p=1
temperature=1
# how_to_cut='凑四句一切'
how_to_cut='按标点符号切'

#0306fix版本有流式输出，输出快了一些，但是中间有停顿
stream=True
audio_save=False
audio_save_file='./output.wav'

sovits_path = r"GPT_SoVITS\pretrained_models\s2G488k.pth"
# sovits_path=r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\xunfeih23r_denoise_e16_s800.pth"
sovits_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise_e25_s575.pth"
gpt_path = r"GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt"
# gpt_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\xunfeih23r_denoise-e15.ckpt"
gpt_path =r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise-e25.ckpt"

audio_save=False
audio_save_file='output.wav'
#---------------------------------------------------#
text_language2='中英混合'
prompt_language2='中英混合'
batch_size=6         # int. batch size for inference
split_bucket=True   # bool. whether to split the batch into multiple buckets.
speed_factor=1.0
ref_text_free=False

if version_choose == 'fast_inference':
    params={'text':text, 'text_lang':text_language2, 
              'ref_audio_path':ref_wav_path, 'prompt_text':prompt_text, 
              'prompt_lang':prompt_language2, 'top_k':top_k, 
              'top_p':top_p, 'temperature':temperature, 
              'text_split_method':how_to_cut, 'batch_size':batch_size, 
              'speed_factor':speed_factor, 'ref_text_free':ref_text_free,
              'split_bucket':split_bucket, 'audio_save':audio_save, 
              'audio_save_file':audio_save_file, 'mygpt_path':gpt_path, 'mysovits_path':sovits_path}
    
elif version_choose == '0306fix':
    params = {'ref_wav_path':ref_wav_path, 'prompt_text':prompt_text, 'prompt_language':prompt_language, 'text':text, 'text_language':text_language, 'sovits_path':sovits_path, 'gpt_path':gpt_path, 'start':start, 'top_k':top_k, 'top_p':top_p, 'temperature':temperature, 'how_to_cut':how_to_cut, 'ref_free':ref_free, 'stream':stream, 'audio_save':audio_save, 'audio_save_file':audio_save_file}


def tts(params):
    # 将标准输出流重定向到空设备
    original_stdout =  sys.stdout
    sys.stdout = open('log_maple.txt', 'w')
    handle(**params)
    sys.stdout = original_stdout
    logging.getLogger().setLevel(logging.WARNING)

tts(params)

Building prefix dict from the default dictionary ...


DEBUG:jieba_fast:Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\maple\AppData\Local\Temp\jieba.cache


DEBUG:jieba_fast:Loading model from cache C:\Users\maple\AppData\Local\Temp\jieba.cache


Loading model cost 0.580 seconds.


DEBUG:jieba_fast:Loading model cost 0.580 seconds.


Prefix dict has been built succesfully.


DEBUG:jieba_fast:Prefix dict has been built succesfully.


In [6]:
#拜拜结束对话
#对话前删除临时录音文件
def delete_files(folder_path):
    file_list = glob.glob(folder_path + '/*')
    for file_path in file_list:
        if os.path.isfile(file_path):
            if '.' in file_path and file_path.split('.')[0].split('/')[-1][:4]=='test':
                os.remove(file_path)

# 调用函数删除文件夹中的文件
audio_file_path = 'test'
delete_files(audio_file_path)

#语音提醒可以对话了
params['text'] ="您好，您现在可以跟我进行对话了！" 
tts(params)

#用户说一次话并得到机器回复即为一轮。用户一轮说话中每有一个相对长的间隔，则将前面的录音分割为独立的一段，这一段视为一波录音。
outfile_name=audio_file_path+'\\test.mp3'
lc =0 #对话轮次，用于录音文件的名称存储
while(True):
    filenum=0 #每轮对话的录音波次
    tag=0 #一轮对话开始的标志
    sign=0 #whisper类是否转换声音了
    lc_result=''#每一轮次的语音输入结果

    while(model_choose !='sherpa_onnx'):
        temp=outfile_name.split('.')
        #当前波次有说话时，若后面说话间隔超过delayTime时进行下一波次录音；若当前波次还没有语音，当时间间隔超过tendure时候结束本轮录音。

        #增加用户的反应时间，3s太短了。
        sign, tag=listen(temp[0]+str(lc)+str('-')+str(filenum)+'.'+temp[1], tag, delayTime=0.8, tendure=5, mindb = 2000)
        # print(tag)
        if sign==1:#一轮对话录音结束的标志,每轮最后一波不转文字
            # if tag==0: #如果第一波录音算是反应时间
            #     continue
            break
        else:
            #将该波次语音转为文字
            # print(1,end='')
            open_audio_file = '/'.join(outfile_name.split('//'))
            open_audio_file=open_audio_file.split('.')
            open_audio_file = open_audio_file[0]+str(lc)+str('-')+str(filenum)+'.'+open_audio_file[1]
            # print(1,end='')
            #---------------有声音，但是不是话语时候，fast_whisper会输出随机的词语。
            if model_choose=='faster_whisper':
                result = fast_whisper_opt(model, open_audio_file, initial_prompt=None)
            #------------------whisper听到声音，但是这些声音是语气词无法翻译成话语时候会出错。
            elif model_choose=='whisper':
                result = whisper_output(model, open_audio_file, initial_prompt=None)
            #当对话是刚开始的时候
            # print(2,end='')
    
            lc_result += result
            # lc_result += ','
            if re.sub(r'[ .。！!?？-]','', result) in {'拜拜', '再见','byebye', 'Byebye' ,'掰掰'}:
                print(result)
                lc=-1
                break

            print(result, end='')
        filenum += 1

    #注意：每轮最后一波录音是空白

    if sign==1 and tag==0:#当用户开始说话但是无语音输入时候，退出整个对话
        print('\n', '*'*40)
        print(' '*5, '您的对话已超时，请重新对话！')
        print('\n', '*'*40)
        break
    #----------------------------------------------------------------------------#
    #shepra_onnx asr模型，快，但是不那么准确
    #输入声音但是无法翻译成话语时候会直接结束对话
    if model_choose == 'sherpa_onnx':
        # print('用户:  请输入你的问题')
        last_result, lc_result = microphone_endpoint.main(args_dict, False, check_recognizer)
        if last_result in {'拜拜'}:
            lc=-1
            result=last_result
        
        if len(lc_result)==0:
            print('\n', '*'*40)
            print(' '*5, '您的对话已超时，请重新对话！')
            print('\n', '*'*40)
            break

    print()
    #每轮的文本互动开始
    print('Bot:',end='')
    if lc==-1:
        print(result)
        break
    lc +=1

    #文本对话
    stream = ollama.chat(
        model=text_text_model,
        messages=[{'role': 'user', 'content': lc_result}],
        stream=True,
    )

    outword=""
    outword3=""

    max_chunk=4#每max_chunk轮播放一次
    chunk_num=1
    for chunk in stream:
        outword3 = chunk['message']['content']
        #打印对话内容
        print(outword3, end='', flush=True)
        outword += outword3
        #语音播放
        if outword3 in {'\n', ',','.','。','，','!','！', '?', '？',':', '：'}:
            #语音合成
            # 将标准输出流重定向到空设备
            #每max_chunk句就读一次
            #chunk_num==max_chunk:

            #每换一次行第一次。
            if outword3=='\n':
                params['text']=outword
                tts(params)
                # engine.say(outword)
                outword=""
                chunk_num=1
            #gemma是一个字一个chunk
            else:
                chunk_num +=1
        # engine.runAndWait()

    #把没凑够max_chunk的话读出来
    if len(outword)!=0:
        params['text']=outword
        tts(params)
    print('####end', end='')
    params['text'] ='我的回答就是这些。'
    tts(params)
    print()

用户：                              
 ****************************************
      您的对话已超时，请重新对话！

 ****************************************
